In [1]:
!ml Anaconda3
!source activate pannot-dev


In [2]:
from transformers import AutoTokenizer
from pannot.model.language_model.pannot_llama import PannotLlamaForCausalLM
import torch



ModuleNotFoundError: No module named 'transformers'

In [ ]:
# Load base LLM
model = PannotLlamaForCausalLM.from_pretrained(
    "/scratch/user/yining_yang/TAMU/PhD/Pannot/local_pretrained_llm/Meta-Llama-3.1-8B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",  # automatic single-device or CPU+GPU placement
)

# Load adapter weights
adapter_weights = torch.load("TAMU/PhD/Pannot/checkpoints/pannot-Meta-Llama-3.1-8B-Instruct-pretrain-v00/mm_projector.bin", map_location="cpu")
model.load_state_dict(adapter_weights, strict=False)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("/scratch/user/yining_yang/TAMU/PhD/Pannot/local_pretrained_llm/Meta-Llama-3.1-8B-Instruct")

# Input prompt
prompt = "Describe the function of this protein."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

print("Has sequence encoder:", hasattr(model, "seq_tower"))
print("Has structure encoder:", hasattr(model, "struc_tower"))

# Optional: check inner modules
print("Seq tower type:", type(model.seq_tower))
print("Struc tower type:", type(model.struc_tower))


# Run generation
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=256,
    do_sample=False,
    temperature=0.7
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))